In [ ]:
import os
import json
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization, add
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, EarlyStopping

In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/My Drive/Deep_Learning_homework/Data'

/content/drive/My Drive/Deep_Learning_homework/Data


In [ ]:
# Veri yolları
csv_path = "train.csv"
json_path = "label_num_to_disease_map.json"
image_folder = "train_images"
# Sınıf adları
with open(json_path, 'r') as f:
    class_names = json.load(f)

In [ ]:
# Veri yollarını yükleyin
df = pd.read_csv(csv_path)

# **Etiket Sayısı hesaplama**

In [ ]:
# Label sütunundaki değerleri say
label_counts = df['label'].value_counts()

# Etiketlerin sayısını yazdır
print("Etiket 0 olanların sayısı:", label_counts.get(0, 0))
print("Etiket 1 olanların sayısı:", label_counts.get(1, 0))
print("Etiket 2 olanların sayısı:", label_counts.get(2, 0))
print("Etiket 3 olanların sayısı:", label_counts.get(3, 0))
print("Etiket 4 olanların sayısı:", label_counts.get(4, 0))

Etiket 0 olanların sayısı: 1087
Etiket 1 olanların sayısı: 2189
Etiket 2 olanların sayısı: 2386
Etiket 3 olanların sayısı: 13158
Etiket 4 olanların sayısı: 2577


In [ ]:
# Her sınıf için resim ve labellı resim sayılarını hesapla
class_counts = {}
for label, class_name in class_names.items():
    class_images = len(df[df['label'] == int(label)])
    labeled_images = class_images
    class_counts[class_name] = {'images': class_images, 'labeled_images': labeled_images}

    print(f"Sınıf: {class_name}, Toplam Resim Sayısı: {class_images}, Etiketli Resim Sayısı: {labeled_images}")

Sınıf: Cassava Bacterial Blight (CBB), Toplam Resim Sayısı: 1087, Etiketli Resim Sayısı: 1087
Sınıf: Cassava Brown Streak Disease (CBSD), Toplam Resim Sayısı: 2189, Etiketli Resim Sayısı: 2189
Sınıf: Cassava Green Mottle (CGM), Toplam Resim Sayısı: 2386, Etiketli Resim Sayısı: 2386
Sınıf: Cassava Mosaic Disease (CMD), Toplam Resim Sayısı: 13158, Etiketli Resim Sayısı: 13158
Sınıf: Healthy, Toplam Resim Sayısı: 2577, Etiketli Resim Sayısı: 2577


In [ ]:
import os

# train.csv dosyasının boyutunu yazdır
csv_size = os.path.getsize(csv_path)
print(f"train.csv dosyasının boyutu: {csv_size} bytes")

# image_folder içindeki resim sayısını hesapla
image_files = os.listdir(image_folder)
num_images = len(image_files)
print(f"{image_folder} klasöründe {num_images} resim bulunmaktadır.")


train.csv dosyasının boyutu: 379681 bytes
train_images klasöründe 17938 resim bulunmaktadır.


In [ ]:
import os

image_folder = "train_images"
image_id = "1000015157.jpg"

img_path = os.path.join(image_folder, image_id)

if os.path.exists(img_path):
    print(f"{image_id} dosyası {image_folder} klasöründe bulunuyor.")
else:
    print(f"{image_id} dosyası {image_folder} klasöründe bulunmuyor.")


1000015157.jpg dosyası train_images klasöründe bulunmuyor.


In [ ]:
import pandas as pd

csv_path = "train.csv"
image_id_to_check = "1000015157.jpg"

# CSV dosyasını oku
df = pd.read_csv(csv_path)

# image_id sütununda belirtilen dosyanın olup olmadığını kontrol et
if image_id_to_check in df['image_id'].values:
    print(f"{image_id_to_check} dosyası train.csv dosyasında bulunuyor.")
else:
    print(f"{image_id_to_check} dosyası train.csv dosyasında bulunmuyor.")


1000015157.jpg dosyası train.csv dosyasında bulunuyor.


In [ ]:
import pandas as pd
import os

# Boş bir DataFrame oluştur
new_df = pd.DataFrame(columns=['image_id', 'label'])

df = pd.read_csv(csv_path)


for image_id, label in zip(df['image_id'], df['label']):
    img_path = os.path.join(image_folder, image_id)

    # Eğer dosya mevcutsa işleme devam et, aksi takdirde atla
    if os.path.exists(img_path):
        # Yeni bir satır oluştur
        new_row = pd.DataFrame({'image_id': [image_id], 'label': [label]})
        # Mevcut DataFrame'e yeni satırı ekle
        new_df = pd.concat([new_df, new_row], ignore_index=True)
# Sonuçları göster
print(new_df)


             image_id label
0       157078263.jpg     3
1      1574893536.jpg     3
2      1575013487.jpg     3
3      1576606254.jpg     0
4      1579761476.jpg     1
...               ...   ...
17933   999068805.jpg     3
17934   999329392.jpg     3
17935   999474432.jpg     1
17936   999616605.jpg     4
17937   999998473.jpg     4

[17938 rows x 2 columns]


In [ ]:
df.shape[0]

21397

In [ ]:
new_df.to_csv('new_train.csv', index=False)

# **Preprocessing**

In [ ]:

# Data and Labels Collection
data = []
labels = []
success_count = 0
skip_count = 0

df = pd.read_csv("new_train.csv")

for image_id, label in zip(df['image_id'], df['label']):
    img_path = os.path.join(image_folder, image_id)

    # Eğer dosya mevcutsa işleme devam et, aksi takdirde atla
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        resized_img = cv2.resize(img, (227, 227))
        data.append(resized_img)
        labels.append(label)
        success_count += 1
    else:
        print(f"Uyarı: {img_path} bulunamadı, bu nedenle atlandı.")
        skip_count += 1

# Convert to NumPy arrays
data = np.array(data)
labels = np.array(labels)

print(f"Başarılı işlenen fotoğraf sayısı: {success_count}")
print(f"Atlanan fotoğraf sayısı: {skip_count}")

Başarılı işlenen fotoğraf sayısı: 17938
Atlanan fotoğraf sayısı: 0


In [ ]:
#Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

num_classes = len(class_names)

# Convert training labels to one-hot encoding
y_train_one_hot = to_categorical(y_train, num_classes)

# Convert testing labels to one-hot encoding
y_test_one_hot = to_categorical(y_test, num_classes)

# **Model Oluşturma**

In [ ]:
def residual_block(input_tensor, filters, stride=(1, 1), shortcut_connection=False):
    shortcut = input_tensor

    # First convolutional layer of the block
    x = Conv2D(filters, (1, 1), strides=stride, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second convolutional layer of the block
    x = Conv2D(filters, (3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Third convolutional layer of the block
    x = Conv2D(filters * 4, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)

    # Input tensorün boyutları output tensor ile uyuşmazsa,
    # boyutları 1x1 Conv ile değiştir
    if shortcut_connection:
        shortcut = Conv2D(filters * 4, (1, 1), strides=stride, padding='same')(input_tensor)
        shortcut = BatchNormalization()(shortcut)

    # 3.Conv layera shortcut ekle
    x = add([x, shortcut])
    x = Activation('relu')(x)

    return x

def resnet50(input_shape=(224, 224, 3), num_classes=5):
    inputs = Input(shape=input_shape)

    # Conv layerı başlat
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # 1. residual blocks group
    x = residual_block(x, filters=64, shortcut_connection=True)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)

    # 2. residual blocks group
    x = residual_block(x, filters=128, stride=(2, 2), shortcut_connection=True)
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=128)

    # 3. residual blocks group
    x = residual_block(x, filters=256, stride=(2, 2), shortcut_connection=True)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=256)

    # 4. residual blocks group
    x = residual_block(x, filters=512, stride=(2, 2), shortcut_connection=True)
    x = residual_block(x, filters=512)
    x = residual_block(x, filters=512)

    # Global average pooling
    x = GlobalAveragePooling2D()(x)

    # Fully connected layer
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)

    return model


In [ ]:
# Build vggnet Model
resnet50_model = resnet50(num_classes=5)

# Print Model Summary
resnet50_model.summary()
# Compile Model
resnet50_model.compile(optimizer= Adam(0.001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])


In [ ]:
checkpoint = ModelCheckpoint("best_vggnet_model_weights.h5",
                             monitor='val_acc',  # Kaydetme kriteri olarak doğruluk metriğini kullanın
                             verbose=1,
                             save_best_only=True,    # Sadece en iyi performansı gösteren ağırlıkları kaydedin
                             mode='max')
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, mode='auto', factor=0.1, min_lr=0.000001)
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

In [ ]:
# Modeli eğit
history=resnet50_model.fit(X_train, y_train_one_hot, epochs=100, batch_size=32, validation_split=0.2, verbose=1,callbacks=[reduce_lr,checkpoint,early_stopping])